## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
## dependencies

import pandas as pd
import requests
import gmaps
import numpy as np

from config import g_key

In [2]:
gmaps.configure(api_key=g_key)
# configure gmaps api key

In [3]:
## import the WeatherPy_Database.csv file

city_data_df = pd.read_csv(r"WeatherPy_Database.csv")
city_data_df.head()

,City,Lat,Lng,Weather Description,Max Temp,Humidity,Wind Speed,Cloudiness,Country
0,Olinda,-8.0089,-34.8553,few clouds,80.330,69,4.12,20,BR
1,Hithadhoo,-0.6000,73.0833,broken clouds,82.670,69,1.88,55,MV
2,Carnarvon,-24.8667,113.6333,overcast clouds,68.378,73,0.90,88,AU
3,Lloydminster,53.2835,-110.0016,broken clouds,62.816,67,5.14,75,CA
4,Nanortalik,60.1432,-45.2371,broken clouds,41.684,72,4.78,81,GL


In [4]:
## prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [5]:
## filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.tail(10)

,City,Lat,Lng,Weather Description,Max Temp,Humidity,Wind Speed,Cloudiness,Country
640,Rey Bouba,8.6724,14.1786,few clouds,79.646,63,0.44,12,CM
643,San Policarpo,12.1791,125.5072,overcast clouds,80.006,81,3.06,100,PH
646,Song Phi Nong,14.2241,100.0209,overcast clouds,80.276,82,2.19,100,TH
647,Palmer,42.1584,-72.3287,clear sky,82.454,43,1.34,7,US
651,Badarganj,25.6740,89.0549,moderate rain,76.334,95,5.32,100,BD
654,Shenzhen,22.5455,114.0683,light rain,79.214,97,1.42,100,CN
656,Sungairaya,0.7000,108.9000,overcast clouds,76.586,90,1.57,96,ID
659,Barcelona,41.3888,2.1590,few clouds,82.616,48,2.57,20,ES
661,Suez,29.9737,32.5263,clear sky,75.578,68,4.25,0,EG
674,Ahuimanu,21.4447,-157.8378,mist,77.774,85,3.60,100,US


In [6]:
## drop any empty rows and create a new DataFrame that doesn’t have empty rows

preferred_cities_df.loc[(preferred_cities_df["City"] == ""), "City"] = np.nan
# searches the "City" column for any missing entries
clean_cities_df = preferred_cities_df.dropna(subset=["City"])


clean_cities_df.tail(10)

,City,Lat,Lng,Weather Description,Max Temp,Humidity,Wind Speed,Cloudiness,Country
640,Rey Bouba,8.6724,14.1786,few clouds,79.646,63,0.44,12,CM
643,San Policarpo,12.1791,125.5072,overcast clouds,80.006,81,3.06,100,PH
646,Song Phi Nong,14.2241,100.0209,overcast clouds,80.276,82,2.19,100,TH
647,Palmer,42.1584,-72.3287,clear sky,82.454,43,1.34,7,US
651,Badarganj,25.6740,89.0549,moderate rain,76.334,95,5.32,100,BD
654,Shenzhen,22.5455,114.0683,light rain,79.214,97,1.42,100,CN
656,Sungairaya,0.7000,108.9000,overcast clouds,76.586,90,1.57,96,ID
659,Barcelona,41.3888,2.1590,few clouds,82.616,48,2.57,20,ES
661,Suez,29.9737,32.5263,clear sky,75.578,68,4.25,0,EG
674,Ahuimanu,21.4447,-157.8378,mist,77.774,85,3.60,100,US


In [7]:
## determines if there are any empty rows

clean_cities_df.isnull().sum()

City                   0
Lat                    0
Lng                    0
Weather Description    0
Max Temp               0
Humidity               0
Wind Speed             0
Cloudiness             0
Country                0
dtype: int64

In [8]:
## create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()
# copy the selected columns over to new dataframe
hotel_df["Hotel Name"] = ""
# create a new empty column "Hotel Name"

hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Olinda,BR,80.330,few clouds,-8.0089,-34.8553,
1,Hithadhoo,MV,82.670,broken clouds,-0.6000,73.0833,
6,Jamestown,US,80.942,clear sky,42.0970,-79.2353,
10,Ormara,PK,81.500,clear sky,25.2088,64.6357,
13,Buchanan,LR,78.188,overcast clouds,5.8808,-10.0467,
15,Puerto Ayora,EC,78.764,overcast clouds,-0.7393,-90.3518,
16,Bathsheba,BB,81.212,broken clouds,13.2167,-59.5167,
18,Jacareacanga,BR,76.964,scattered clouds,-6.2222,-57.7528,
23,Kavieng,PG,81.824,broken clouds,-2.5744,150.7967,
24,Hilo,US,84.272,broken clouds,19.7297,-155.0900,


In [9]:
## this bit will retrieve a list of hotels related to our city data

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}
# set parameters to search for hotels with 5000 meters.

for index, row in hotel_df.iterrows():
# iterate through the hotel DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # get latitude and longitude from DataFrame

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # set up the base URL for the Google Directions API to get JSON data
    
    hotels = requests.get(base_url, params=params).json()
    # make request and retrieve the JSON data from the search 
   
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    # get the first hotel from the results and store the name, if a hotel isn't found skip the city     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.tail(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
640,Rey Bouba,CM,79.646,few clouds,8.6724,14.1786,
643,San Policarpo,PH,80.006,overcast clouds,12.1791,125.5072,Agapita M. Nicart
646,Song Phi Nong,TH,80.276,overcast clouds,14.2241,100.0209,แฮปปี้เฮ้าส์ รีสอร์ท
647,Palmer,US,82.454,clear sky,42.1584,-72.3287,Wedgewood Motel
651,Badarganj,BD,76.334,moderate rain,25.6740,89.0549,Raushon
654,Shenzhen,CN,79.214,light rain,22.5455,114.0683,Grand Skylight
656,Sungairaya,ID,76.586,overcast clouds,0.7000,108.9000,Rumah Kuku Moimoi Shelly
659,Barcelona,ES,82.616,few clouds,41.3888,2.1590,Axel Hotel Barcelona
661,Suez,EG,75.578,clear sky,29.9737,32.5263,Green House
674,Ahuimanu,US,77.774,mist,21.4447,-157.8378,Paradise Bay Resort


In [11]:
## drop the rows where there is no Hotel Name

hotel_df.loc[(hotel_df["Hotel Name"] == ""), "Hotel Name"] = np.nan
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
# searches the "City" column for any missing entries

In [12]:
clean_hotel_df.tail(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
639,Cockburn Town,TC,84.164,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
643,San Policarpo,PH,80.006,overcast clouds,12.1791,125.5072,Agapita M. Nicart
646,Song Phi Nong,TH,80.276,overcast clouds,14.2241,100.0209,แฮปปี้เฮ้าส์ รีสอร์ท
647,Palmer,US,82.454,clear sky,42.1584,-72.3287,Wedgewood Motel
651,Badarganj,BD,76.334,moderate rain,25.6740,89.0549,Raushon
654,Shenzhen,CN,79.214,light rain,22.5455,114.0683,Grand Skylight
656,Sungairaya,ID,76.586,overcast clouds,0.7000,108.9000,Rumah Kuku Moimoi Shelly
659,Barcelona,ES,82.616,few clouds,41.3888,2.1590,Axel Hotel Barcelona
661,Suez,EG,75.578,clear sky,29.9737,32.5263,Green House
674,Ahuimanu,US,77.774,mist,21.4447,-157.8378,Paradise Bay Resort


In [13]:
clean_hotel_df.to_csv(r"WeatherPy_Vacation.csv", index_label="City_ID")
# export the City_Data into a csv

In [14]:
## using the template add city name, the country code, the weather description and maximum temperature for the city

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp:.2f} °F</dd>
</dl>
"""
# <dl> tag is a description list, <dt> is a nested description list under <dl>, <dd> defines term or names <dt> tag

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# get the data from each row and add it to the formatting template and store the data in a list

locations = clean_hotel_df[["Lat", "Lng"]]
# get the latitude and longitude from each row and store in a new DataFrame

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))